In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64tz_dtype
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
# import duckdb 
import warnings
# import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution
import seaborn as sns
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
#Run the following line in the event the virtual environment is not being listed
# (naarni_env) root@jupyter-0:/home/notebook/tasks# python -m ipykernel install --user --name=my_project_env --display-name="naarni_venv"

# jupyter nbconvert --to python 6.bcs_tms_analysis.ipynb \
#     --TemplateExporter.exclude_markdown=True \
#     --TemplateExporter.exclude_output_prompt=True \
#     --TemplateExporter.exclude_input_prompt=True

In [3]:
sys.path.append('..')
from common import db_operations
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,execute_query

In [4]:
conn = connect_to_trino()
query = f"""
select 
    id,count(*) as ttl_data
FROM 
    facts_prod.can_parsed_output_all
where 
    dt = DATE('2025-12-05')
group by 1
order by 2 desc
"""
df = execute_query(conn, query)
df.head()

2025-12-10 13:49:12 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-10 13:49:12 - INFO - ✅ STEP 1: Connected to Trino
2025-12-10 13:49:12 - INFO - ⚙️ Executing query...
2025-12-10 13:49:14 - INFO - ✅ Query executed successfully with results!


In [6]:
import pandas as pd

opt_df = pd.read_csv("optimized_attribute_dictionary.csv")

# Build a Python dict: {column_name: optimized_type}
opt_map = dict(zip(opt_df["Column"], opt_df["optimized_type"]))

# opt_map

In [7]:
def build_select_with_cast(opt_map):
    select_exprs = []
    for col, dtype in opt_map.items():
        if dtype == "UNMAPPED":  
            # Keep original type
            select_exprs.append(f'"{col}"')
        else:
            select_exprs.append(f'CAST("{col}" AS {dtype}) AS "{col}"')
    return ",\n".join(select_exprs)

In [8]:
def build_trino_query(table, id_val, date_val, opt_map):
    select_clause = build_select_with_cast(opt_map)
    query = f"""
    SELECT
        {select_clause}
    FROM {table}
    WHERE id = '{id_val}'
      AND dt = DATE('{date_val}')
      AND motor_torque < 60000
    """
    return query


In [9]:
def fetch_device_day(id_val, date_val, lite=False):
    use_map = lite_map if lite else opt_map
    query = build_trino_query("facts_prod.can_parsed_output_all",id_val,date_val,use_map)
    
    print("Running optimized query...")
    return execute_query(conn, query)

In [10]:
def enforce_dtypes(df, opt_map):
    for col, dtype in opt_map.items():

        if col not in df.columns:
            continue
        
        # 1. SMALLINT → Int16
        if dtype == "SMALLINT":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int16")
        
        elif dtype == "TINYINT":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int8")
        
        # 2. DECIMAL(4,1) / (6,1) / (6,3) / (12,3)
        elif dtype.startswith("DECIMAL"):
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("float32")
        
        # 3. TIMESTAMP
        elif dtype == "TIMESTAMP(6)":
            df[col] = pd.to_datetime(df[col], errors="coerce")

        # 4. UNMAPPED
        else:
            try:
                df[col] = pd.to_numeric(df[col], errors="ignore")
            except:
                pass

    return df


In [11]:
columns_to_drop = opt_df[
    # opt_df["optimized_type"].isin(["DECIMAL(4,3)", "SMALLINT"]) &
    # opt_df["Column"].str.contains("pack_cellvoltage|temperature|vin_digit|spn|phase|date|rank", case=False)
    opt_df["Column"].str.contains("date|rank", case=False)
]["Column"].tolist()

lite_map = {c: t for c, t in opt_map.items() if c not in columns_to_drop}

In [12]:
df = fetch_device_day("16", "2025-11-27", lite=True)
df = enforce_dtypes(df, opt_map)
df.head()

2025-12-10 13:49:14 - INFO - ⚙️ Executing query...


Running optimized query...


2025-12-10 13:49:25 - INFO - ✅ Query executed successfully with results!


,id,timestamp,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,lowpressureoilpumpfaultcode,lowvoltagedcaclifesignal,batterycoolingstate,batterycoolanttemperature,odometerreading,brakepedalpos,ebsredwarningsignal,abs_ebsamberwarningsignal,insulation_status,insulation__value,lifesignalofinsulation,output_phase_voltage,output_phase_currant,motor_rpm,motor_torque,motor_temperature,igbt_temperature,mcuself_teststatus,side_door_panel_open_alarm,ignitionstatus,brakingsystemmallfunction_alarm,parking_brake_status,front_air_pressure,rear_air_pressure,brake_pad_worn_out_alarm,guna_dcp_temperature,guna_dcm_temperature,gunb_dcp_temperature,gunb_dcm_temperature,gunc_dcp_temperature,gunc_dcm_temperature,batterypackovervoltagealarm,monomerovervoltagealarm,polehightemperaturealarm,batteryhightemperaturealarm,hightempalarmofcharginggun,monomerundervoltagealarm,batterypackundervoltagealarm,temperaturedifferencealarm,singlevoltagedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,sochighalarm,soclowalarm,bms_fault_code,gun_connection_status,maxavailableshorttermcharging,maxavailableshorttimedischarge,chargingcontactor2positive,chargingcontactor2negative,chargingcontactor1positive,batterytotalnegativecontactor,chargingcontactor1negative,pack_negative_contactors_status,bat_soc,bat_voltage,total_battery_current,bmslifesignal,steering_pump_enable_command,air_compressor_enable_command,dcdc_enable_command,vcuversioninformation,distancetoempty,accelarationpedal,brake_pedal,vcuself_teststatus,systeminterlockstate,vehiclereadycondition,vehicle_speed_vcu,gear_position,autoholdunenable,powertrain_failure_alarm,vcu_fault_code,vehiclecontrollerlife,cellmax_voltagecellnumber,cell_max_voltage,cellminvoltagecellnumber,cell_min_voltage,pack1_cellmax_temperature,pack1_maxtemperature_cell_number,pack1_cell_min_temperature,pack1_celltemperature_cellnumber,fourinonelifesignal,fourinoneself_teststatus,prechargedcontactorstate,mainmotorcontactor1state,accessorycontactorstatus,pre_charge_status,fiveinone_faultcode,airpumpcondition,air_compreesor_temperature,airpumpdcaclifesignal,dcdc_statusandfailure,dcdc_out_put_currant,dcdc_out_put_volatge,dcdcbusvoltage,dcdclifesignal,oilpumpcondition,steering_pump_rpm,steeringcontroller_failurealarm,lift_lower_normal_request,kneeling_request,th_management_sys_working_mode,soh,charging_contactor_3_positive,charging_contactor_3_negative,charger_voltage,charger_current,chrargecurrent_request,evcc_error,chargingerror_bms,vehicle_operation_mode,pack_temperature1,pack_temperature2,pack_temperature3,pack_temperature4,pack_temperature5,pack_temperature6,pack_temperature7,pack_temperature8,pack_temperature9,pack_temperature10,pack_temperature11,pack_temperature12,pack_temperature13,pack_temperature14,pack_temperature15,pack_temperature16,pack_temperature17,pack_temperature18,pack_temperature19,pack_temperature20,pack_temperature21,pack_temperature22,pack_temperature23,pack_temperature24,pack_temperature25,pack_temperature26,pack_temperature27,pack_temperature28,pack_temperature29,pack_temperature30,pack_temperature31,pack_temperature32,pack_temperature33,pack_temperature34,pack_temperature35,pack_temperature36,pack_temperature37,pack_temperature38,pack_temperature39,pack_temperature40,pack_temperature41,pack_temperature42,pack_temperature43,pack_temperature44,pack_temperature45,pack_temperature46,pack_temperature47,pack_temperature48,pack_temperature49,pack_temperature50,pack_temperature51,pack_temperature52,pack_temperature53,pack_temperature54,pack_temperature55,pack_temperature56,pack_temperature57,pack_temperature58,pack_temperature59,pack_temperature60,pack_temperature61,pack_temperature62,pack_temperature63,pack_temperature64,pack_temperature65,pack_temperature66,pack_temperature67,pack_temperature68,pack_temperature69,pack_temperature70,pack_temperature71,pack_temperature72,pack_temperature73,pack_temperature74,pack_temperature75,pack_temperature76,pack_temperature77,pack_temperature78,pack_

## Make provision to filter columns and their ranges in a csv rather than in code so that it can be integrated within the query itself
## do this within the existing optimized_attibutes csv and then use case statements within trino for guard rails

In [19]:
# df.info(verbose=True)
# null_chk = round((df.isnull().sum()*100.0)/len(df),2)
# null_chk[null_chk<100].sort_values()

In [15]:
# Filter out pack_cellvoltage_* columns
filtered_cols = cols[
    ~(
        cols['Column'].str.startswith("pack_cellvoltage_") |
        cols['Column'].str.startswith("pack_temperature") |
        cols['Column'].str.startswith("vin_digit")
    )
]
# filtered_cols = cols[cols['Column'].str.startswith("pack_temperature")]

filtered_cols.head()
column_list = filtered_cols['Column'].tolist()
select_clause = ", ".join([f'"{c}"' for c in column_list])
print(select_clause)


In [16]:
# 5. Final query (example with your filters)
final_query = f"""
SELECT
    distinct brakepedalpos
FROM facts_prod.can_parsed_output_all
WHERE id IN ('16')
  AND dt = DATE('2025-12-05')
"""

print(final_query)  # Optional, to inspect generated SQL

# 6. Execute the query
df_final = execute_query(conn, final_query)

df_final.head()

In [17]:
# df_final.brakepedalpos.describe()